In this notebook, we train a model using the orientation data of people during fitness exercises. The model tries to predict whether the person is in a vertical or a horizontal position.

In [1]:
import sys
import os
import subprocess
import zipfile
import numpy as np

from oodles import Framework
from oodles import Signal
from oodles import monitor
from oodles import ModelSignal, AnnotationMethod, Anomaly

from dataset import input_to_dataset_transformation, read_json, write_json, KpsDataset
from pushup_signal import pushup_signal
from contextlib import redirect_stdout

import tensorflow as tf
import joblib
import json

Download dataset from remote

In [2]:
data_dir = "data"
remote_url = "https://oodles-dev-training-data.s3.amazonaws.com/data.zip"
orig_training_file = 'data/training_data.json'
if not os.path.exists(data_dir):
    try:
        file_downloaded_ok = subprocess.check_output("wget " + remote_url, shell=True)
    except:
        print("Could not load training data")
    with zipfile.ZipFile("data.zip", 'r') as zip_ref:
        zip_ref.extractall("./")

    full_training_data = read_json(orig_training_file)
    np.random.seed(1)
    np.random.shuffle(full_training_data)
    reduced_training_data = full_training_data[0:1000]
    write_json(orig_training_file, reduced_training_data)

In [3]:
real_world_test_cases = 'data/real_world_testing_data.json'
data_save_fold_name = 'oodles_smart_data'
golden_testing_file = 'data/golden_testing_data.json'
annotation_args = {'master_file': 'data/master_annotation_data.json'}

# Defining the egde-case signal
pushup_edge_case = Signal("Pushup", pushup_signal)

### Training with Logistic Regression (LR)

In [4]:
from model_logistic_regression import get_accuracy_lr, train_model_lr
train_model_lr('data/training_data.json', 'version_0')

Training on:  data/training_data.json  which has  1000  data-points
Model saved at:  trained_models_lr/version_0


Next, we evaluate the model on our golden testing dataset.

In [5]:
get_accuracy_lr(golden_testing_file, 'version_0')

Evaluating on  15731  data-points


0.8586231008836056

We observe that the testing accuracy of the model is quite low. On investigating further, we realize that it is because all pushup signals are being classified as "vertical" orientation. Next, we will define the oodles config with edge-case check for Pushup signals and also pass our training and evaluation arguments

In [6]:
cfg = {
    # Define your signal to identify edge cases
    "checks": [{
        'type': Anomaly.EDGE_CASE, 
        "signal_formulae": pushup_edge_case
    }],

    # Connect training pipeline to annotate data and retrain the model
    "training_args": {
        "data_transformation_func": input_to_dataset_transformation,  
        "annotation_method": {"method": AnnotationMethod.MASTER_FILE, "args": annotation_args}, 
        "training_func": train_model_lr, 
        "fold_name": data_save_fold_name,  
        "orig_training_file": orig_training_file,  
    },

    # Connect evaluation pipeline to test retrained model against original model
    "evaluation_args": {
        "inference_func": get_accuracy_lr,
        "golden_testing_dataset": golden_testing_file,
        "metrics_to_check": ['accuracy']
    }
}

In [7]:
framework_lr = Framework(cfg)

@monitor(framework_lr)
def model_predict(args):
    return args['model'].predict(args['kps'])

In [8]:
testing_dataset = KpsDataset(real_world_test_cases, normalization=True)
X_test, y_test, id = testing_dataset.load_x_y_from_data()
pred_classes = []
model = joblib.load("trained_models_lr/" + 'version_0')
for i,elem in enumerate(X_test):
    preds = model_predict({"model": model, "kps": elem.reshape(1, -1), "id": [id[i]]})
    if framework_lr.version > 1:
        # Retrain only once
        break

50  edge-cases collected out of  76  inferred samples
100  edge-cases collected out of  145  inferred samples
150  edge-cases collected out of  224  inferred samples
200  edge-cases collected out of  296  inferred samples
250  edge-cases collected out of  367  inferred samples
Kicking off re-training
251 data-points selected out of 371
Training on:  oodles_smart_data/1/training_dataset.json  which has  2255  data-points
Model saved at:  trained_models_lr/version_1
Model retraining done...
Generating comparison report...
Training on:  data/training_data.json  which has  1000  data-points
Trained model exists. Skipping training again.
Evaluating on  15731  data-points
Evaluating on  15731  data-points
---------------------------------------------
---------------------------------------------
Old model accuracy:  0.8586231008836056
Retrained model accuracy (ie 251 smartly collected data-points added):  0.9597609815014939
---------------------------------------------
----------------------

In the comparison report above, we can see how oodles improved the model performance by detecting edge-cases and retraining the model under-the-hood. Further, Oodles is agnostic to the model type and training functions. To illustrate this, we again train our orientation classification model, but this time with Deep Neural Networks.

### Training using Deep Neural Network

In [9]:
from model_dnn import get_accuracy_dnn, train_model_dnn
train_model_dnn('data/training_data.json', 'version_0')

Training on:  data/training_data.json  which has  1000  data-points


2022-11-17 17:20:58.363717: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
3/3 [==============================] - 1s 174ms/step - loss: 143.7527 - binary_accuracy: 0.2604
Epoch 2/10
3/3 [==============================] - 1s 181ms/step - loss: 129.5267 - binary_accuracy: 0.2826
Epoch 3/10
3/3 [==============================] - 1s 189ms/step - loss: 122.6065 - binary_accuracy: 0.2812
Epoch 4/10
3/3 [==============================] - 1s 167ms/step - loss: 116.1137 - binary_accuracy: 0.2943
Epoch 5/10
3/3 [==============================] - 1s 162ms/step - loss: 97.9835 - binary_accuracy: 0.3385
Epoch 6/10
3/3 [==============================] - 0s 167ms/step - loss: 92.0371 - binary_accuracy: 0.3008
Epoch 7/10
3/3 [==============================] - 1s 161ms/step - loss: 87.5087 - binary_accuracy: 0.3372
Epoch 8/10
3/3 [==============================] - 1s 166ms/step - loss: 80.9741 - binary_accuracy: 0.3333
Epoch 9/10
3/3 [==============================] - 1s 166ms/step - loss: 72.0179 - binary_accuracy: 0.3802
Epoch 10/10
3/3 [=========================

Next, we get the model accuracy on testing dataset, which is again low due to misclassification of Pushup signals.

In [10]:
get_accuracy_dnn(golden_testing_file, 'version_0')

Evaluating on  15731  data-points
492/492 [==============================] - 0s 383us/step


0.2280846735744708

Update the Oodles config with new training workflows and checks. Let's also add a check for edge-cases when model confidence is low.

In [11]:
# Whenever model confidence is <0.8, identify it as an edge-case 
low_conf_edge_case = Signal(ModelSignal.BINARY_ENTROPY_CONFIDENCE, 
                is_model_signal=True, extra_args={'conf_threshold': 0.8})

cfg['checks'][0].update({"signal_formulae": (pushup_edge_case | low_conf_edge_case)})
cfg['training_args'].update({'training_func': train_model_dnn})
cfg['evaluation_args'].update({'inference_func': get_accuracy_dnn})

In [12]:
framework_dnn = Framework(cfg)

@monitor(framework_dnn)
def model_predict(args):
    with open('evaluation_logs.txt', 'w') as f:
        with redirect_stdout(f):
            return args['model'].predict(args['kps'])

Deleting the folder:  oodles_smart_data


In [13]:
model_dir = 'trained_models_dnn/'
model_save_name = 'version_0'
real_world_dataset = KpsDataset(
    real_world_test_cases, batch_size=1, shuffle=False, augmentations=False, is_test=True
)
model = tf.keras.models.load_model(model_dir + model_save_name)
for i,elem in enumerate(real_world_dataset):
    preds = model_predict({"model": model, "kps": elem[0]["kps"], "id": elem[0]["id"]})
    if framework_dnn.version > 1:
        # Retrain only once
        break

50  edge-cases collected out of  186  inferred samples
100  edge-cases collected out of  372  inferred samples
150  edge-cases collected out of  575  inferred samples
200  edge-cases collected out of  799  inferred samples
250  edge-cases collected out of  958  inferred samples
Kicking off re-training
251 data-points selected out of 960
Training on:  oodles_smart_data/1/training_dataset.json  which has  2255  data-points
Epoch 1/10
8/8 [==============================] - 1s 162ms/step - loss: 75.3781 - binary_accuracy: 0.5205
Epoch 2/10
8/8 [==============================] - 1s 164ms/step - loss: 64.3419 - binary_accuracy: 0.5068
Epoch 3/10
8/8 [==============================] - 1s 165ms/step - loss: 58.2991 - binary_accuracy: 0.4897
Epoch 4/10
8/8 [==============================] - 1s 172ms/step - loss: 49.5159 - binary_accuracy: 0.4985
Epoch 5/10
8/8 [==============================] - 1s 169ms/step - loss: 40.2772 - binary_accuracy: 0.5161
Epoch 6/10
8/8 [=============================